In [1]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are available:", gpus)
else:
    print("No GPUs available, using CPU.")

TensorFlow version: 2.10.0
No GPUs available, using CPU.


In [4]:
import tensorflow as tf  # Import TensorFlow library for deep learning
from tensorflow.keras.models import Sequential  # Sequential model for building neural networks layer by layer
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D  # Import necessary layers
from tensorflow.keras.callbacks import TensorBoard
import pickle  # Import the pickle library for loading data
import numpy as np  # Import NumPy for numerical operations
import time  # Import time for timestamp generation
import os  # Import os for handling file paths

# Limit CPU usage
os.environ["OMP_NUM_THREADS"] = "2"        # Set the number of OpenMP threads
os.environ["TF_NUM_INTRAOP_THREADS"] = "2" # Number of threads used for intra-op parallelism
os.environ["TF_NUM_INTEROP_THREADS"] = "1" # Number of threads used for inter-op parallelism

# Load preprocessed image data and labels from pickle files
X = pickle.load(open("X.pickle", "rb"))  # Load feature data (images)
y = pickle.load(open("y.pickle", "rb"))  # Load labels (categories)

# Normalize the pixel values of images to a range of 0 to 1
X = X / 255.0  

# Convert y to a NumPy array to ensure it's in the correct format for training
y = np.array(y)

# Verify that the number of samples in X matches the number of labels in y
assert X.shape[0] == y.shape[0], "Number of samples in X and y must match."

# Configure TensorFlow to use GPU appropriately
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Set layer variables.
dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            # Create a unique name for the TensorBoard logs using the current timestamp
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer,int(time.time()))
            print(NAME)
            # Initialize TensorBoard callback with the correct log directory
            log_dir = os.path.join("logs", NAME)
            tensorboard = TensorBoard(log_dir=log_dir)
            
            # Initialize a Sequential model
            model = Sequential()
            
            # Add convolutional layers and pooling layers
            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2, 2)))
            
            # Flatten the output from the convolutional layers to feed into the dense layers
            model.add(Flatten())

            # Add a fully connected (dense) layer
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
           
            
            # Output layer for binary classification
            model.add(Dense(1))
            model.add(Activation("sigmoid"))
            
            # Compile the model with specified parameters
            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
            
            # Train the model using the training data and TensorBoard callback for logging
            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1734087613
Epoch 1/10
1638/1638 [==============================] - 256s 153ms/step - loss: 0.5262 - accuracy: 0.7263 - val_loss: 0.4305 - val_accuracy: 0.8034
Epoch 2/10
1638/1638 [==============================] - 256s 157ms/step - loss: 0.3708 - accuracy: 0.8346 - val_loss: 0.3256 - val_accuracy: 0.8521
Epoch 3/10
1638/1638 [==============================] - 253s 154ms/step - loss: 0.2963 - accuracy: 0.8731 - val_loss: 0.3002 - val_accuracy: 0.8713
Epoch 4/10
1638/1638 [==============================] - 257s 157ms/step - loss: 0.2395 - accuracy: 0.8988 - val_loss: 0.1951 - val_accuracy: 0.9193
Epoch 5/10
1638/1638 [==============================] - 253s 154ms/step - loss: 0.1912 - accuracy: 0.9225 - val_loss: 0.2069 - val_accuracy: 0.9096
Epoch 6/10
1638/1638 [==============================] - 258s 158ms/step - loss: 0.1463 - accuracy: 0.9422 - val_loss: 0.1480 - val_accuracy: 0.9388
Epoch 7/10
1638/1638 [==============================] - 266s 162ms/step - los

INFO:tensorflow:Assets written to: 64x3-CNN.model\assets


INFO:tensorflow:Assets written to: 64x3-CNN.model\assets
